# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# loading csv to create df

df = pd.read_csv("Output_CSV/cities.csv")
df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chichli,22.8333,78.8167,71.58,20,0,1.03,IN,1614786819
1,Punta Arenas,-53.1500,-70.9167,57.20,54,40,6.91,CL,1614786518
2,Barrow,71.2906,-156.7887,-14.80,76,90,12.66,US,1614786550
3,Dingle,10.9995,122.6711,75.20,88,18,6.46,PH,1614786567
4,Busselton,-33.6500,115.3333,71.01,88,100,5.01,AU,1614786407


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configuring maps
gmaps.configure(api_key=g_key)

# using lat & lng as locations and humidity as weight
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]

In [6]:
#adding heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrow_city_df=df[(df["Max Temp"]>70)&(df["Max Temp"]<80)&(df["Wind Speed"]<10)& (df["Cloudiness"]==0)].dropna()
narrow_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chichli,22.8333,78.8167,71.58,20,0,1.03,IN,1614786819
18,Hermanus,-34.4187,19.2345,73.99,72,0,5.01,ZA,1614786531
22,Carnarvon,-24.8667,113.6333,75.20,83,0,5.75,AU,1614786507
79,Salalah,17.0151,54.0924,78.80,73,0,4.61,OM,1614786675
103,Peravurani,10.3000,79.1833,75.45,69,0,8.61,IN,1614786847
336,Narasannapeta,18.4167,84.0500,73.38,83,0,1.77,IN,1614786914
346,Hambantota,6.1241,81.1185,78.80,83,0,8.05,LK,1614786524
347,Mathbaria,22.2876,89.9593,70.86,94,0,5.35,BD,1614786917
356,Palauig,15.4350,119.9045,78.40,79,0,4.14,PH,1614786920
359,Alipur,29.3846,70.9116,75.56,12,0,7.58,PK,1614786921


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create new variable
hotel_df = df[["City","Country", "Lat", "Lng"]].copy()

#add column 
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Chichli,IN,22.8333,78.8167,
1,Punta Arenas,CL,-53.1500,-70.9167,
2,Barrow,US,71.2906,-156.7887,
3,Dingle,PH,10.9995,122.6711,
4,Busselton,AU,-33.6500,115.3333,
...,...,...,...,...,...
557,Yeniseysk,RU,58.4497,92.1703,
558,Skeldon,GY,5.8833,-57.1333,
559,Itháki,GR,38.3642,20.7185,
560,Jacareacanga,BR,-6.2222,-57.7528,


In [9]:
# set parameters

params = {
    "radius": 5000,
    "type": "hotel",
    "key":g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# plot markers on heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))